In [21]:
import pandas as pd

In [53]:
df = pd.read_csv('../grams/sg_de.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams

In [87]:
# list of ~94k US babynames to remove (hello, john), (james, bond)
names = pd.read_csv("../names.csv", names=['name'])['name'].tolist()

In [55]:
german_filter = ['ich', 'du', 'sie', 'mir', 'wir', 'ihr', 'uns', 'euch', 'die', 'das', 'der', 'den', 'er', 'und', '!', '/', 'ein', 'eine', 'einer', 'einem', 'mein', 'dein', 'nicht']

In [56]:
dutch_filter = ['ik', 'jij', 'hij', 'zij', 'wij', 'we', 'jullie', 'hun', 'mij', 'me', 'mijn', 'haar', 'hem', '!', '/', 'een', 'de', 'het', 'wie', 'wat', 'waar', 'hoe', 'niet']

In [57]:
l = names + german_filter

In [58]:
df = \
    df[ \
    ~df.word1.isin(l) & \
    ~df.word2.isin(l) & \
    ~df.word1.str.isnumeric() & \
    ~df.word2.str.isnumeric() &\
    ~(df.word1 == df.word2)
    ]

In [59]:
df

,frequency,word1,word2
0,4693,??ber,auf
10,1013,einen,nennen
12,130,sturm,aus
16,167,roten,sind
19,9757,ist,wo
22,33485,wissen,ist
23,105,leben,erwacht
24,1711,es,gro??er
26,134,es,erforschen
35,1567,oben,unten


In [60]:
df.count()['frequency']

288233

In [61]:
wc_de = pd.read_csv("../word_counts/wc_de1.csv", lineterminator='\n', names=['word', 'word_frequency'])

In [62]:
df = df.merge(wc_de, left_on='word1', right_on='word', how='left') \
.merge(wc_de, left_on='word2', right_on='word', how='left') \
.drop(['word_x', 'word_y'], axis=1) \
.rename(index=str, columns={"word_frequency_x": "word_1_frequency", "word_frequency_y": "word_2_frequency"})

In [63]:
df['normalised_frequency'] = df['frequency'] / (df['word_1_frequency']+df['word_2_frequency']) # divide skipgram freq. by wordcount freq.

In [66]:
df = df.sort_values(by='frequency', ascending=False)

In [67]:
df

,frequency,word1,word2,word_1_frequency,word_2_frequency,normalised_frequency
11827,607621,es,ist,3196654,4236656,0.081743
155583,468309,was,ist,2387393,4236656,0.070698
87769,306065,es,zu,3196654,1991560,0.058992
80587,260714,ist,es,4236656,3196654,0.035074
157062,256849,um,zu,583880,1991560,0.099730
177138,209711,ist,zu,4236656,1991560,0.033671
93733,182353,tut,leid,240465,194850,0.418899
1521,161627,wenn,es,971053,3196654,0.038781
70514,161274,es,war,3196654,988962,0.038531
27975,157324,wie,es,1309258,3196654,0.034915
